In [1]:
import pandas as pd
import numpy as np
from chembl_webresource_client.new_client import new_client

In [43]:
GRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF gtpase code\RF_molecules_Newfeatures_GTPase_KRas.csv")
CRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF G12C Code\RF_molecules_Newfeatures_G12C.csv")
DRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF G12D code\RF_molecules_Newfeatures_G12D.csv")
GNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN GTPase Code\\NN_molecules_Newfeatures_GTPase_KRAS.csv")
CNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN G12C Code\\NN_molecules_Newfeatures_G12C.csv")
DNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN G12D Code\\NN_molecules_Newfeatures_G12D(1).csv")

In [44]:
# use this to convert the Neural Network pIC50 values to floats in the dataframe
GNN["Predicted_Value"] = GNN["Predicted_Value"].str.extract(r'np\.float32\((.*?)\)')[0].astype(float)
CNN["Predicted_Value"] = CNN["Predicted_Value"].str.extract(r'\[(.*?)\]')[0].astype(float)
DNN["Predicted_Value"] = DNN["Predicted_Value"].str.extract(r'\[(.*?)\]')[0].astype(float)

# and also drop the Unnamed: 0 column
GNN = GNN.drop(["Unnamed: 0"], axis=1)
CNN = CNN.drop(["Unnamed: 0"], axis=1)
DNN = DNN.drop(["Unnamed: 0"], axis=1)
CNN.head()

,chembl_id,Predicted_Value
0,CHEMBL1200728,10.190047
1,CHEMBL2107067,7.760003
2,CHEMBL1371,7.448368
3,CHEMBL492,5.872513
4,CHEMBL878,5.800352


In [46]:
# drop column frequency for Random Forest
GRF = GRF.drop(["Frequency"], axis=1)
DRF = DRF.drop(["Frequency"], axis=1)
CRF = CRF.drop(["Frequency"], axis=1)

# rename them to fit Neural Network convention
GRF.rename(columns={"ChEMBL ID": "chembl_id", "Avg Predicted Value": "Predicted_Value"}, inplace=True)
DRF.rename(columns={"ChEMBL ID": "chembl_id", "Avg Predicted Value": "Predicted_Value"}, inplace=True)
CRF.rename(columns={"ChEMBL ID": "chembl_id", "Avg Predicted Value": "Predicted_Value"}, inplace=True)
GRF.head()

,chembl_id,Predicted_Value
0,CHEMBL4594350,8.106427
1,CHEMBL2359966,7.497036
2,CHEMBL2361370,7.419709
3,CHEMBL3989958,7.372708
4,CHEMBL3707183,7.497036


In [47]:
df_set = {
  "Gtpase Random Forest": GRF,
  "G12C Random Forest": CRF,
  "G12D Random Forest": DRF,
  "Gtpase Neural Network": GNN,
  "G12C Neural Network": CNN,
  "G12D Neural Network": DNN
}

In [48]:
list = []
for key, value in df_set.items():
    list.append([key, " "," "])
    df = value
    for i in df["chembl_id"]:
        molecule = new_client.molecule
        m1 = molecule.filter(chembl_id=i).only(
            ['molecule_chembl_id', 'pref_name', 'molecule_structures'])
        targets = pd.DataFrame.from_dict(m1)
        #print(targets['molecule_chembl_id'][0])
        if targets['molecule_chembl_id'][0] == i:
            list.append([i,targets['pref_name'][0],float(df.loc[df["chembl_id"]== i ]["Predicted_Value"].values[0])])
            # print(i,targets['pref_name'][0],df[df["chembl_id"]== i ]["Predicted_Value"].values[0])
    list.append([" ", " "," "])

In [49]:
list

[['Gtpase Random Forest', ' ', ' '],
 ['CHEMBL4594350', 'ADAGRASIB', 8.106427136975098],
 ['CHEMBL2359966', 'QUININE SULFATE', 7.497035616770629],
 ['CHEMBL2361370', 'PENBUTOLOL SULFATE', 7.419709268156769],
 ['CHEMBL3989958', 'IVOSIDENIB', 7.372708472127514],
 ['CHEMBL3707183', 'QUINIDINE SULFATE', 7.497035616770629],
 ['CHEMBL4297091', 'CODEINE SULFATE', 7.425687232920642],
 ['CHEMBL3187985', 'APOMORPHINE HYDROCHLORIDE', 7.257022141325173],
 ['CHEMBL2103744', 'MORPHINE SULFATE', 7.2252363553078975],
 ['CHEMBL3183409', 'APALUTAMIDE', 7.217008092870295],
 ['CHEMBL4535757', 'SOTORASIB', 7.173295985258044],
 [' ', ' ', ' '],
 ['G12C Random Forest', ' ', ' '],
 ['CHEMBL1201174', 'SITAGLIPTIN PHOSPHATE', 7.608019155290329],
 ['CHEMBL5315049', 'SITAGLIPTIN FUMARATE', 7.513760295461277],
 ['CHEMBL1200588', 'CLINDAMYCIN HYDROCHLORIDE', 7.478496654781677],
 ['CHEMBL1779710', 'EVOGLIPTIN', 7.457149776386765],
 ['CHEMBL535650', 'MEFLOQUINE HYDROCHLORIDE', 7.455809946116003],
 ['CHEMBL3989551', '

In [50]:
final = pd.DataFrame(list)

In [51]:
final.head()

,0,1,2
0,Gtpase Random Forest,,
1,CHEMBL4594350,ADAGRASIB,8.106427
2,CHEMBL2359966,QUININE SULFATE,7.497036
3,CHEMBL2361370,PENBUTOLOL SULFATE,7.419709
4,CHEMBL3989958,IVOSIDENIB,7.372708


In [52]:
final.to_excel("list_for_Risha.xlsx", index=False)